In [2]:
import json
from datetime import datetime
from openai import OpenAI
from playsound import playsound
from PIL import Image
from urllib.parse import urlparse
from urllib.request import urlopen
from IPython.display import Image
from IPython.core.display import HTML

from os.path import exists
import dotenv

dotenv.load_dotenv()

True

In [2]:
client = OpenAI()

In [3]:
messages = []

In [4]:
from carparks.types import Point


# def get_tools():
#     """Get the list of tools that can be used in the chatbot"""
#     return [
#         {
#             "type": "function",
#             "function": {
#                 "name": "get_current_date_and_time",
#                 "description": "Get the current date and time",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {},
#                 },
#             }
#         },
#         {
#             "type": "function",
#             "function": {
#                 "name": "create_image",
#                 "description": "Create an image from a prompt",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {
#                         "prompt": {
#                             "type": "string",
#                             "description": "The prompt to use to generate the image",
#                         },
#                     },
#                 },
#             }
#         },
#     ]
def get_tools():
    """Get the list of tools that can be used in the chatbot"""
    return [
        # {
        #     "type": "function",
        #     "function": {
        #         "name": "get_current_location",
        #         "description": "Get current location latitude and longitude",
        #         "parameters": {
        #             "type": "object",
        #             "properties": {
        #                 "address": {
        #                     "type": "string",
        #                     "description": "The address to get the latitude and longitude",
        #                     "default":"Siu Sai Wan, Hong Kong"
        #                 }
        #             },
        #             "required": ["address"]
        #         },
        #     }
        # },
        {
            "type": "function",
            "function": {
                "name": "get_nearest_car_parks",
                "description": "Fetch car parks base on a set of criteria",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "current_location": {
                            "type": "object",
                            "description": "The current location latitude and longitude",
                            "properties": {
                                "latitude": {
                                    "type": "number",
                                    "description": "The latitude of the current location",
                                    "default": 22.261515,
                                },
                                "longitude": {
                                    "type": "number",
                                    "description": "The longitude of the current location",
                                    "default": 114.249452
                                },
                            },
                            "required": ["latitude", "longitude"]
                        }

                    },
                    "required": ["current_location"]
                },
            },
        }
    ]

In [5]:
def get_current_date_and_time(args):
    return datetime.now().strftime("%b-%d-%Y %H:%M:%S, %A")


def create_image_wrapper(args):
    return create_image(args["prompt"])

In [6]:


def speak(message, ):
    speech_file_path = "./speech.mp3"
    client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message
    ).stream_to_file(speech_file_path)
    playsound(speech_file_path)

In [7]:
def resize_image(image_url):
    threshold = 255
    image = Image.open(image_url if is_local(image_url) else urlopen(image_url))
    width, height = image.size
    if width > threshold or height > threshold:
        if width > height:
            new_width = threshold
            new_height = int(height * threshold / width)
        else:
            new_height = threshold
            new_width = int(width * threshold / height)

        resized_image = image.resize((new_width, new_height))
        resized_image.save('./resized_image.jpg')
        return './resized_image.jpg'

In [8]:
def is_local(url):
    url_parsed = urlparse(url)
    if url_parsed.scheme in ('file', ''):  # Possibly a local file
        return exists(url_parsed.path)
    return False

In [9]:
def get_current_location(args):
    return json.dumps({
        "address": "Siu Sai Wan, Hong Kong",
        "location": {
            "latitude": 22.261515,
            "longitude": 114.249452
        }
    })

In [10]:
from utils import load_data_from_file, calculate_distance


def get_nearest_car_parks(args):
    print('args', args)

    current_point = Point(lat=args['current_location']['latitude'], lng=args['current_location']['longitude'])
    carparks = load_data_from_file('./carparks/hk.json')

    selected_car_parks = []

    for carpark in carparks:
        carpark_point = Point(lat=carpark['geocoding']['geometry']['location']['lat'],
                              lng=carpark['geocoding']['geometry']['location']['lng'])
        distance = calculate_distance(current_point, carpark_point)
        selected_car_parks.append({
            "name": carpark["name"],
            "address": carpark["address"],
            "distance": distance,
        })

    selected_car_parks.sort(key=lambda x: x['distance'])

    return json.dumps(selected_car_parks[:5])





968853.5466871759


In [11]:
# def get_available_functions():
#     return {
#         "get_current_date_and_time": get_current_date_and_time,
#         "create_image": create_image_wrapper,
#     }

def get_available_functions():
    return {
        # "get_current_location": get_current_location,
        "get_nearest_car_parks": get_nearest_car_parks,
    }

In [12]:
def ask(message: str, image_url=None):
    model = "gpt-4-1106-preview"
    # model = "gpt-4-1106-preview" if image_url is None else "gpt-4-vision-preview"

    user_message = {
        "role": "user",
        "content": message,
    } if image_url is None else {
        "role": "user",
        "content": [
            {"type": "text", "text": message},
            {"type": "image_url", "image_url": {"url": image_url}},
        ],
    }

    print(model, user_message)

    messages.append(user_message)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=get_tools(),
        tool_choice="auto")

    response_message = response.choices[0].message

    tool_calls = response_message.tool_calls

    if tool_calls:
        available_functions = get_available_functions()  #

        # {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [ChatCompletionMessageToolCall(id='call_Pds6T6VQ7WH6CZG478GlhHkx', function=Function(arguments='{}', name='get_current_date_and_time'), type='function')]}

        messages.append({
            "content": None,
            "role": "assistant",
            "tool_calls": [{"id": call.id, "function": call.function.__dict__, "type": call.type} for call in
                           tool_calls],
        })

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(function_args)

            print('function_response', function_response)

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

            print('messages', messages)
        reply_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )
        reply_message = reply_response.choices[0].message.content
    else:
        reply_message = response.choices[0].message.content

    messages.append({
        "role": "assistant",
        "content": reply_message,
    })
    print(reply_message)

    # speak(reply_message)

In [13]:
def create_image(prompt: str):
    """Using DALL-E 3 to create image from prompt. And return the image url"""
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )

    print("response", response.data[0].revised_prompt)

    image_url = response.data[0].url

    print("image_url", image_url)

    Image(url=image_url)

    return f"Created image can be found with the following image: {image_url}"


In [14]:
ask("Can you give me the addresses of the top 2 closest car park?")

gpt-4-1106-preview {'role': 'user', 'content': 'Can you give me the addresses of the top 2 closest car park?'}
args {'current_location': {'latitude': 22.261515, 'longitude': 114.249452}}
function_response [{"name": "\u5c0f\u897f\u7063\u90a8\u4e00\u671f\u505c\u8eca\u5834 Siu Sai Wan Estate Phase 1 (\u5c0f\u897f\u7063\u5ee3\u5834 Siu Sai Wan Plaza)", "address": "\u9999\u6e2f\u5c0f\u897f\u7063\u5c0f\u897f\u7063\u905310\u865f", "distance": 142.4542245544509}, {"name": "\u5c0f\u897f\u7063\u90a8\u4e09\u671f\u505c\u8eca\u5834 Siu Sai Wan Estate Phase 3", "address": "\u9999\u6e2f\u5c0f\u897f\u7063\u5c0f\u897f\u7063\u905310\u865f", "distance": 142.4542245544509}, {"name": "\u85cd\u7063\u5ee3\u5834\u505c\u8eca\u5834 Island Resort Mall (\u85cd\u7063\u534a\u5cf6 Island Resort)", "address": "\u9999\u6e2f\u67f4\u7063\u5c0f\u897f\u7063\u905328\u865f", "distance": 555.6050739713276}, {"name": "\u5b89\u529b\u5de5\u696d\u4e2d\u5fc3\u505c\u8eca\u5834 Honour Industrial Centre", "address": "\u9999\u6e2f\u6

In [15]:
ask("I don't like these 2, can you give me another 2?")

gpt-4-1106-preview {'role': 'user', 'content': "I don't like these 2, can you give me another 2?"}
args {'current_location': {'latitude': 22.261515, 'longitude': 114.249452}}
function_response [{"name": "\u5c0f\u897f\u7063\u90a8\u4e00\u671f\u505c\u8eca\u5834 Siu Sai Wan Estate Phase 1 (\u5c0f\u897f\u7063\u5ee3\u5834 Siu Sai Wan Plaza)", "address": "\u9999\u6e2f\u5c0f\u897f\u7063\u5c0f\u897f\u7063\u905310\u865f", "distance": 142.4542245544509}, {"name": "\u5c0f\u897f\u7063\u90a8\u4e09\u671f\u505c\u8eca\u5834 Siu Sai Wan Estate Phase 3", "address": "\u9999\u6e2f\u5c0f\u897f\u7063\u5c0f\u897f\u7063\u905310\u865f", "distance": 142.4542245544509}, {"name": "\u85cd\u7063\u5ee3\u5834\u505c\u8eca\u5834 Island Resort Mall (\u85cd\u7063\u534a\u5cf6 Island Resort)", "address": "\u9999\u6e2f\u67f4\u7063\u5c0f\u897f\u7063\u905328\u865f", "distance": 555.6050739713276}, {"name": "\u5b89\u529b\u5de5\u696d\u4e2d\u5fc3\u505c\u8eca\u5834 Honour Industrial Centre", "address": "\u9999\u6e2f\u67f4\u7063\u6

In [5]:
playsound("./speech.mp3")